In [1]:
def chose_input(mlist):
    chose_index = (mlist['crawl_mark'] == 'False') & (mlist['cleaned'].notnull())
    return (mlist.loc[chose_index].index[0], mlist.loc[chose_index, 'cleaned'].iloc[0])

def google_search_crawler(input_index, input_keyword, user_agents):
    # input_keyword = '-4 fun'
    # 組合URL
    url = "https://www.google.com.tw/search?q=" + input_keyword + "&start=0"
    user_agent = user_agents[random.randint(0, 9)]
    # 請求
    res = requests.get(url, user_agent)
    # 解析
    soup = BeautifulSoup(res.text, "html.parser")
    search_text = soup.find_all("div", class_="g") # 剝一層
    # 開始訪問細項
    title_list = list() # title
    context_list = list() # context
    cite_list = list() # cite
    for result in search_text:
        try: # title
            title_list.append(result.find(class_='r').find('a').text)
        except:
            title_list.append(None)
        try: # context
            context_list.append(result.find(class_='st').text)
        except:
            context_list.append(None)
        try: # cite
            cite_list.append(result.find('cite').text)
        except:
            try: # google map沒有cite，要特別辨識
                if result.find('img').get_attribute_list('id')[0] == 'lu_map':
                    cite_list.append('map.google.com')
                else:
                    cite_list.append(None)
            except:
                cite_list.append(None)
    input_text = [input_keyword for i in range(len(title_list))] # 產生等長的輸入文字
    input_row = [input_index for i in range(len(title_list))] # 產生等長的原始index
    '''
    # 如果右邊有wiki
    search_text[-1] # 基本在最後面
    search_text[-1].find(class_='FSP1Dd').text # title
    search_text[-1].find('span').text # context
    tempp = re.sub('(\\n|\.\.\.)', '', search_text[-1].get_text(separator=',')) #其他人也搜尋了
    tempp = re.sub('.*也搜尋了,', '', tempp)
    '''
    # 創建矩陣，long表格
    result = pd.DataFrame({'mlist_index': input_row, 'search_text': input_text,
                           'result_title': title_list, 'result_context': context_list,
                           'result_cite':cite_list})
    result = result[['mlist_index', 'search_text', 'result_cite', 'result_title', 'result_context']]
    
    # 後修正，因為有些問題要直接用網頁結構拆有難度，等結果出來再修正
    # 有"圖片搜尋結果"的頁面，把cite改成img.google
    result.loc[result['result_title'].str.match('.*圖片搜尋結果$', na=False), 'result_cite'] = 'img.google.com'
    # 右側有wiki的頁面，爬下來
    if result.iloc[-1, 2:].isnull().all():
        wiki_part = search_text[-1]
        try:
            result.iloc[-1, 2] = 'wikipedia.org'
            result.iloc[-1, 3] = wiki_part.find(class_='FSP1Dd').text
            result.iloc[-1, 4] = wiki_part.find('span').text.replace(' 維基百科', '')
            
            # 該block下方有"其他人也搜尋了..."
            tempp = re.sub('(\\n|\.\.\.)', '', search_text[-1].get_text(separator=',')) # 其他人也搜尋了
            tempp = re.sub('.*搜尋了[\u4e00-\u9fa5]*,', '', tempp)
            simi_temp = pd.DataFrame({'mlist_index': input_index, 'search_text': input_keyword,
                                      'result_cite': 'similar', 'result_title': '知識圖譜相似',
                                      'result_context': tempp}, index=[0])
            result = pd.concat([result, simi_temp], axis=0, ignore_index=True)
        except:
            pass
    # 最下方的其他人也搜尋了
    try:
        others_search_string = soup.find_all('p', class_='aw5cc')
        others_search_string = [temp.text for temp in others_search_string]
        simi_temp = pd.DataFrame({'mlist_index': input_index, 'search_text': input_keyword,
                                      'result_cite': 'similar', 'result_title': '相似搜尋字串',
                                      'result_context': ','.join(others_search_string)}, index=[0])
        result = pd.concat([result, simi_temp], axis=0, ignore_index=True)
    except:
        pass
    # sleep
    time.sleep(1 + random.uniform(0, 1) + random.uniform(0, 1))
    return result

In [2]:
import pandas as pd
import requests
import time
import random
from bs4 import BeautifulSoup
import re
# 請求頭
user_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20130406 Firefox/23.0', \
      'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:18.0) Gecko/20100101 Firefox/18.0', \
      'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/533+ \
      (KHTML, like Gecko) Element Browser 5.0', \
      'IBM WebExplorer /v0.94', 'Galaxy/1.0 [en] (Mac OS X 10.5.6; U; en)', \
      'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)', \
      'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14', \
      'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) \
       Version/6.0 Mobile/10A5355d Safari/8536.25', \
      'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) \
       Chrome/28.0.1468.0 Safari/537.36', \
      'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0; TheWorld)']
# load mlist
mlist = pd.read_csv('D:/iima/進行中/貼標工程/data/crawler/clean_mlist_for_gsearch_20190417.csv',
                engine='python', encoding='UTF-8')
mlist['crawl_mark'] = mlist['crawl_mark'].astype('str')

In [3]:
mlist.head()
mlist.shape

(237129, 4)

In [4]:
type(mlist['crawl_mark'].iloc[0])

str

In [5]:
chose_input(mlist)

(0, '富邦人壽')

In [ ]:
# 執行爬蟲
all_result = pd.DataFrame(columns=['mlist_index', 'search_text', 'result_cite', 'result_title', 'result_context'])
a = 0
b = 0
for i in range(len(mlist)):
    input_tuple = chose_input(mlist)

    try:
        all_result = all_result.append(google_search_crawler(input_tuple[0], input_tuple[1], user_agents))
        mlist.iloc[input_tuple[0], 2] = 'True'
    except:
        mlist.iloc[input_tuple[0], 2] = 'error'
        print(input_tuple[1])
    a += 1
    if (a > 99) or (i == (len(mlist) -1)):
        b += a
        a = 0
        print('%s rows saved!' % b)
        # 存出
        all_result.to_csv('D:/iima/進行中/貼標工程/data/crawler/output_google_search_result_20190419.csv', index=False, encoding='UTF-8', mode='a')
        mlist.to_csv('D:/iima/進行中/貼標工程/data/crawler/output_clean_mlist_for_gsearch_20190419.csv', index=False, encoding='UTF-8')
        all_result = pd.DataFrame(columns=['mlist_index', 'search_text', 'result_cite', 'result_title', 'result_context'])

100 rows saved!
200 rows saved!
300 rows saved!
400 rows saved!
500 rows saved!
600 rows saved!
700 rows saved!
800 rows saved!
900 rows saved!
1000 rows saved!
1100 rows saved!
1200 rows saved!
1300 rows saved!
1400 rows saved!
1500 rows saved!
1600 rows saved!
1700 rows saved!
1800 rows saved!
1900 rows saved!
2000 rows saved!
2100 rows saved!
2200 rows saved!
2300 rows saved!
2400 rows saved!
2500 rows saved!
2600 rows saved!
2700 rows saved!
2800 rows saved!
2900 rows saved!
3000 rows saved!
3100 rows saved!
3200 rows saved!
3300 rows saved!
3400 rows saved!
3500 rows saved!
3600 rows saved!
3700 rows saved!
3800 rows saved!
3900 rows saved!
4000 rows saved!
4100 rows saved!
4200 rows saved!
4300 rows saved!
4400 rows saved!
4500 rows saved!
4600 rows saved!
4700 rows saved!
4800 rows saved!
4900 rows saved!
5000 rows saved!
5100 rows saved!
5200 rows saved!
5300 rows saved!
5400 rows saved!
5500 rows saved!
5600 rows saved!
5700 rows saved!
5800 rows saved!
5900 rows saved!
藍新金流-m

In [8]:
mlist['crawl_mark'].value_counts()
[i for i in range(0, 10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]